In [1]:
from sklearn.cluster import KMeans

In [2]:
from ghmm import *

In [3]:
import numpy as np

In [4]:
import pickle

In [5]:
grfKMeans = pickle.load( open( "grfKMeans.model", "rb" ) )

In [6]:
grfDataset = pickle.load( open( "grfDataset_standing", "rb"))

In [7]:
grfDataset.shape


(1994, 10)

In [8]:
grfKMeans.predict(grfDataset[1800:1800+20])

array([40,  6, 40, 40,  6, 40, 40,  6,  6, 40, 60,  6, 40, 36, 36, 40, 40,
        6,  6,  6], dtype=int32)

In [9]:
count = 0

In [10]:
i = 0

In [11]:
N = 20

In [12]:
l = 0

In [13]:
seqList = []

In [14]:
seqLengths = []

In [15]:
while True:
    block = grfDataset[i:i+N]
    seq = grfKMeans.predict(block)
    l = seq.size
    seqList.append(seq)
    
    count = count + 1
    if l != N:
        break
    i = i + N
    
        

In [16]:
len(seqList)

100

In [17]:
#seqList

# 123

In [18]:
npA = np.random.dirichlet((1, 2, 3, 4, 5),size=5)

In [19]:
npB = np.random.dirichlet((1,2,3,4,5,6,7,8,9,10,
                           11,12,13,14,15,16,17,18,19,20,
                           21,22,23,24,25,26,27,28,29,30,
                           31,32,33,34,35,36,37,38,39,40,
                           41,42,43,44,45,46,47,48,49,50,
                           51,52,53,54,55,56,57,58,59,60,
                           61,62,63,64),size=5)

In [20]:
npA.shape

(5, 5)

In [21]:
npB.shape

(5, 64)

In [22]:
A=npA.tolist()

In [23]:
B=npB.tolist()

In [24]:
Pi=[1.0/5] * 5

In [25]:
sigma = IntegerRange(0,64)

In [26]:
hmmModel = HMMFromMatrices(sigma, DiscreteDistribution(sigma), A, B, Pi)

In [27]:
trainSet = SequenceSet(sigma, seqList)

In [28]:
hmmModel.baumWelch(trainSet, 100)

In [29]:
testSeq=EmissionSequence(sigma, [40,  6, 40, 40,  6, 40, 40,  6,  6, 40, 60,  6, 40, 36, 36, 40, 40,
        6,  6,  6] )

In [30]:
print(hmmModel.viterbi(testSeq))

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0], -31.77359208832549)


# 參考: 
http://ghmm.sourceforge.net/documentation.html



In [31]:
#print(hmmModel)

In [32]:
#import pickle

In [33]:
#pickle.dump(hmmModel, open("ghmmModel_standing", "wb"))

In [34]:
#help('ghmm')

In [35]:
#hmmModel.cmodel

In [36]:
testSeq1=EmissionSequence(sigma, [ 5, 22, 16,  2, 59, 10, 20, 20, 59,  4, 56, 43, 26, 57,  5, 54, 54,
        5, 16, 44] )

In [37]:
print(hmmModel.viterbi(testSeq1))

GHMM ghmm.py:148 - viterbi.c:ghmm_dmodel_viterbi(328): state_seq_index = -1, t = 19


([-1], 1.0)


In [ ]:
# backup and restore functions

In [40]:
def myGHMMDump(sigma_, hmmModel_, fileName_):
    A_=hmmModel_.asMatrices()[0]
    B_=hmmModel_.asMatrices()[1]
    Pi_=hmmModel_.asMatrices()[2]
    
    HMM_PARAMS_=[sigma_, A_, B_, Pi_]
    pickle.dump(HMM_PARAMS_, open(fileName_, "wb"))

In [52]:
def myGHMMLoad(fileName_):
    HMM_PARAMS_ = pickle.load(open(fileName_, "rb"))
    sigma_=HMM_PARAMS_[0]
    A_=HMM_PARAMS_[1]
    B_=HMM_PARAMS_[2]
    Pi_=HMM_PARAMS_[3]
    
    hmmModel_ = HMMFromMatrices(sigma_, DiscreteDistribution(sigma_), A_, B_, Pi_)
    return [sigma_, hmmModel_]

In [53]:
# backup

In [42]:
myGHMMDump(sigma, hmmModel, "ghmmModel_standing")

In [50]:
# restore

In [54]:
[neverMind, hmmModel1] = myGHMMLoad("ghmmModel_standing")

In [55]:
# test

In [56]:
print(hmmModel1.viterbi(testSeq))

([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 4, 0, 0, 0, 0, 0], -31.77359208832549)


In [1]:
help('ghmm')

Help on module ghmm:

NAME
    ghmm - @mainpage GHMM - an open source library for Hidden Markov Models (HMM)

FILE
    /home/kkuei/anaconda3/envs/rospy/lib/python2.7/site-packages/ghmm.py

DESCRIPTION
    HMMs are stochastic models which encode a probability density over
    sequences of symbols. These symbols can be discrete letters (A,C,G and
    T for DNA; 1,2,3,4,5,6 for dice), real numbers (weather measurement
    over time: temperature) or vectors of either or the combination
    thereof (weather again: temperature, pressure, percipitation).
    
    @note
    We will always talk about emissions, emission sequence and so
    forth when we refer to the sequence of symbols. Another name
    for the same object is observation resp. observation sequence.
    
    A simple model with a fair and one unfair coin can be created as follows
    
    >> fair = [0.5, 0.5]
    >> loaded = [0.9, 0.1]
    >> A = [[0.9, 0.1], [0.3, 0.7]]
    >> pi = [0.9, 0.1]
    >> B = [fair, loaded]
    >> si